# 🚀 胸部 X 光 DINOv2 - 突破 90% 完全自動化訓練

**當前最佳**: 87.574% → **目標**: 90%+ (預期 89.5-91.5%)

**策略**: DINOv2 Vision Transformer (142M 圖片預訓練)

---

## ⚡ 快速開始

### 必須設置（開始前）
1. ✅ **啟用 A100 GPU**: Runtime → Change runtime type → GPU → A100
2. ✅ **準備 Kaggle API**: 下載 `kaggle.json` (Kaggle → Account → Create API Token)

### 執行選項
- **選項 A (推薦)**: 快速驗證 - 只訓練 Fold 0 (2 小時) → 驗證效果
- **選項 B**: 完整訓練 - 所有 5 Folds (8-10 小時) → 目標 90%+

---

## 📊 資料流程
```
GitHub (代碼) → Colab SSD
     ↓
Kaggle (數據 3.4GB) → Colab SSD
     ↓
訓練數據 → A100 GPU (40GB)
     ↓
DINOv2 模型 → 預測 → Kaggle 提交
```

---

**預計時間**: 
- 環境設置: 5-10 分鐘
- Fold 0 訓練: 1.5-2 小時 (A100)
- 5-Fold 完整: 7-10 小時 (A100)


## 1️⃣ 環境檢查與 GPU 設置

In [ ]:
import torch
import os

print("="*70)
print("🔍 環境檢查")
print("="*70)

# Python 版本
import sys
print(f"Python: {sys.version.split()[0]}")

# PyTorch 版本
print(f"PyTorch: {torch.__version__}")

# GPU 檢查
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"\n✅ GPU: {gpu_name}")
    print(f"✅ GPU Memory: {gpu_memory:.1f} GB")
    
    # 推薦配置
    if "A100" in gpu_name:
        batch_size = 32
        print(f"\n🚀 檢測到 A100！使用優化配置:")
        print(f"   - Batch Size: {batch_size}")
        print(f"   - 預計訓練時間: 1.5-2 小時/fold")
    elif gpu_memory > 14:
        batch_size = 16
        print(f"\n✅ 檢測到 {gpu_memory:.0f}GB GPU")
        print(f"   - Batch Size: {batch_size}")
        print(f"   - 預計訓練時間: 2-3 小時/fold")
    else:
        batch_size = 8
        print(f"\n⚠️ GPU 記憶體較小")
        print(f"   - Batch Size: {batch_size}")
        print(f"   - 預計訓練時間: 3-4 小時/fold")
else:
    print("\n❌ 錯誤: 沒有檢測到 GPU！")
    print("請確認: Runtime → Change runtime type → Hardware accelerator → GPU")
    raise RuntimeError("No GPU detected")

print("\n" + "="*70)
print("✅ 環境檢查完成")
print("="*70)

## 2️⃣ 安裝依賴套件

In [ ]:
%%capture
# 靜默安裝，避免過多輸出

# 安裝核心套件
!pip install -q timm>=0.9.0 pyyaml kaggle

# 驗證安裝
import timm
print(f"✅ timm version: {timm.__version__}")

# 檢查 DINOv2 模型
dinov2_models = [m for m in timm.list_models() if 'dinov2' in m]
print(f"✅ DINOv2 models available: {len(dinov2_models)}")
print(f"   Target model: vit_base_patch14_dinov2.lvd142m")

## 3️⃣ 設置 Kaggle API

In [ ]:
from google.colab import files
import os

print("📤 請上傳 kaggle.json 文件...")
print("   (從 Kaggle → Account → Create New API Token 下載)")
print()

uploaded = files.upload()

# 設置 Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 驗證
print("\n驗證 Kaggle API...")
!kaggle competitions list 2>&1 | head -3

print("\n✅ Kaggle API 設置成功！")

## 4️⃣ Clone GitHub 倉庫並切換目錄

In [ ]:
import os

# GitHub 倉庫 URL
REPO_URL = "https://github.com/thc1006/nycu-CSIC30014-LAB3.git"
REPO_NAME = "nycu-CSIC30014-LAB3"

print(f"📦 Clone GitHub 倉庫: {REPO_URL}")

if os.path.exists(REPO_NAME):
    print(f"   倉庫已存在，拉取最新代碼...")
    !cd {REPO_NAME} && git pull
else:
    print(f"   正在 clone...")
    !git clone {REPO_URL}

# 切換到專案目錄
%cd {REPO_NAME}

# 驗證關鍵文件
print("\n驗證關鍵文件:")
!ls -lh train_dinov2_breakthrough.py 2>/dev/null && echo "✅ 訓練腳本" || echo "❌ 訓練腳本缺失"
!ls -lh data/fold_0.csv 2>/dev/null && echo "✅ Fold 數據" || echo "⚠️ Fold 數據需要生成"

print("\n✅ 倉庫準備完成")

## 5️⃣ 下載 Kaggle 競賽數據

In [ ]:
print("📥 下載 Kaggle 競賽數據 (約 3-4 GB)...")
print("   預計時間: 3-5 分鐘\n")

# 下載數據
!kaggle competitions download -c cxr-multi-label-classification

# 解壓到 data 目錄
print("\n📦 解壓數據...")
!unzip -q cxr-multi-label-classification.zip -d data/
!rm cxr-multi-label-classification.zip

# 驗證數據
print("\n驗證數據集:")
import os
train_count = len([f for f in os.listdir('data/train_images') if f.endswith(('.jpg', '.jpeg', '.png'))])
val_count = len([f for f in os.listdir('data/val_images') if f.endswith(('.jpg', '.jpeg', '.png'))])
test_count = len([f for f in os.listdir('data/test_images') if f.endswith(('.jpg', '.jpeg', '.png'))])

print(f"✅ 訓練影像: {train_count} (預期 2718)")
print(f"✅ 驗證影像: {val_count} (預期 679)")
print(f"✅ 測試影像: {test_count} (預期 1182)")

if train_count != 2718 or val_count != 679 or test_count != 1182:
    print("\n⚠️ 警告: 數據集數量不符，請檢查下載是否完整")
else:
    print("\n✅ 數據集下載完成且數量正確！")

## 6️⃣ 準備 K-Fold 訓練數據

In [ ]:
import pandas as pd
import os

print("📊 準備 K-Fold 訓練數據...\n")

# 檢查 Fold CSV 是否存在（應該在 Git 倉庫中）
if os.path.exists('data/fold_0.csv'):
    print("✅ Fold CSV 已存在於 Git 倉庫中")
    
    # 為每個 fold 創建 train/val 分割
    print("\n創建 Fold 訓練/驗證分割...")
    
    # 讀取原始數據
    train_df = pd.read_csv('data/train_data.csv')
    val_df = pd.read_csv('data/val_data.csv')
    all_df = pd.concat([train_df, val_df], ignore_index=True)
    
    for fold in range(5):
        # 讀取 fold 資訊
        fold_df = pd.read_csv(f'data/fold_{fold}.csv')
        
        # 分割 train/val
        train_mask = fold_df['fold'] != fold
        val_mask = fold_df['fold'] == fold
        
        fold_train = all_df[train_mask].copy()
        fold_val = all_df[val_mask].copy()
        
        # 設置影像目錄（統一使用相對路徑）
        # DINOv2 訓練腳本會自動處理路徑
        fold_train['source_dir'] = 'data/train_images'
        fold_val['source_dir'] = 'data/val_images'
        
        # 保存
        fold_train.to_csv(f'data/fold{fold}_train.csv', index=False)
        fold_val.to_csv(f'data/fold{fold}_val.csv', index=False)
        
        print(f"  Fold {fold}: Train={len(fold_train):4d}, Val={len(fold_val):3d}")
    
    print("\n✅ Fold 數據準備完成！")
    
else:
    print("❌ 錯誤: 找不到 Fold CSV 文件")
    print("   請確保 Git 倉庫包含 data/fold_0.csv ~ data/fold_4.csv")
    raise FileNotFoundError("Fold CSV files not found")

## 7️⃣ 選項 A: 快速驗證 - 訓練 Fold 0

**建議**: 先跑 Fold 0 驗證效果 (1.5-2 小時)，如果 Val F1 ≥ 88%，再跑完整 5-Fold

In [ ]:
# Fold 0 快速驗證
FOLD = 0
EPOCHS = 35

# 根據 GPU 自動選擇 batch size
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
if "A100" in torch.cuda.get_device_name(0):
    BATCH_SIZE = 32
elif gpu_memory > 14:
    BATCH_SIZE = 16
else:
    BATCH_SIZE = 8

print("="*70)
print(f"🚀 開始訓練 Fold {FOLD} (快速驗證)")
print("="*70)
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"預計時間: {'1.5-2 小時 (A100)' if BATCH_SIZE >= 32 else '2-3 小時'}")
print("="*70)
print()

# 訓練命令
!python train_dinov2_breakthrough.py \
    --fold {FOLD} \
    --epochs {EPOCHS} \
    --batch_size {BATCH_SIZE} \
    --img_size 518 \
    --lr 3e-5 \
    --output_dir outputs/dinov2_colab

print("\n" + "="*70)
print("✅ Fold 0 訓練完成！")
print("="*70)

## 8️⃣ 檢查 Fold 0 結果並決策

In [ ]:
import re
import os

print("📊 分析 Fold 0 訓練結果...\n")

# 檢查模型是否存在
model_path = 'outputs/dinov2_colab/fold0/best.pt'
if os.path.exists(model_path):
    model_size = os.path.getsize(model_path) / 1e6
    print(f"✅ 模型已保存: {model_path} ({model_size:.1f} MB)")
else:
    print(f"❌ 模型不存在: {model_path}")

# 讀取訓練日誌
log_file = 'outputs/dinov2_colab/fold0/training.log'
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        log_content = f.read()
    
    # 提取最佳 Val F1
    f1_matches = re.findall(r'Val F1: ([\d.]+)%', log_content)
    if f1_matches:
        best_f1 = max([float(x) for x in f1_matches])
        
        print(f"\n{'='*70}")
        print(f"📈 Fold 0 最佳驗證 F1: {best_f1:.2f}%")
        print(f"{'='*70}\n")
        
        # 決策建議
        if best_f1 >= 88:
            print("✅ 🎉 結果優秀！強烈建議繼續完整 5-Fold 訓練")
            print("\n下一步:")
            print("  1. 執行下面的 Cell (完整 5-Fold 訓練)")
            print("  2. 預計 7-10 小時完成所有訓練")
            print("  3. 預期測試 F1: 89.5-91.5% (突破 90% 目標！)")
        elif best_f1 >= 86:
            print("⚠️ 結果良好，但略低於預期 88%")
            print("\n建議:")
            print("  1. 可以嘗試繼續完整訓練")
            print("  2. 或調整超參數後重新訓練 Fold 0")
        else:
            print("❌ 結果不理想，需要檢查")
            print("\n可能問題:")
            print("  1. 檢查數據是否正確加載")
            print("  2. 檢查是否有訓練錯誤")
            print("  3. 查看完整日誌: !tail -50 outputs/dinov2_colab/fold0/training.log")
    else:
        print("⚠️ 無法從日誌提取 F1 分數")
        print("請手動檢查日誌: !tail -100 outputs/dinov2_colab/fold0/training.log")
else:
    print(f"⚠️ 找不到訓練日誌: {log_file}")
    print("請檢查訓練是否成功完成")

## 9️⃣ 選項 B: 完整訓練 - 所有 5 Folds

**警告**: 需要 7-10 小時 (A100) 或 10-15 小時 (T4)

**建議**: 只有在 Fold 0 結果 ≥ 88% 時執行

In [ ]:
import time

EPOCHS = 35

# 自動選擇 batch size
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
if "A100" in torch.cuda.get_device_name(0):
    BATCH_SIZE = 32
elif gpu_memory > 14:
    BATCH_SIZE = 16
else:
    BATCH_SIZE = 8

print("="*70)
print("🚀 開始完整 5-Fold 訓練")
print("="*70)
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"預計總時間: {'7-10 小時 (A100)' if BATCH_SIZE >= 32 else '10-15 小時 (T4)'}")
print("="*70)
print()

start_time = time.time()

for fold in range(5):
    fold_start = time.time()
    
    print(f"\n{'='*70}")
    print(f"訓練 Fold {fold}/4")
    print(f"{'='*70}\n")
    
    !python train_dinov2_breakthrough.py \
        --fold {fold} \
        --epochs {EPOCHS} \
        --batch_size {BATCH_SIZE} \
        --img_size 518 \
        --lr 3e-5 \
        --output_dir outputs/dinov2_colab
    
    fold_time = (time.time() - fold_start) / 60
    total_time = (time.time() - start_time) / 60
    
    print(f"\n✅ Fold {fold} 完成！")
    print(f"   本 Fold 耗時: {fold_time:.1f} 分鐘")
    print(f"   累計耗時: {total_time:.1f} 分鐘")
    
    if fold < 4:
        remaining = fold_time * (4 - fold)
        print(f"   預估剩餘時間: {remaining:.1f} 分鐘 ({remaining/60:.1f} 小時)")

total_time = (time.time() - start_time) / 60
print(f"\n" + "="*70)
print(f"🎉 所有 5 Folds 訓練完成！")
print(f"總耗時: {total_time:.1f} 分鐘 ({total_time/60:.1f} 小時)")
print("="*70)

## 🔟 生成 5-Fold 集成預測並提交

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from tqdm.auto import tqdm
import timm
import os

print("🔮 生成 5-Fold 集成預測...\n")

# 測試集 Dataset
class TestDataset(Dataset):
    def __init__(self, csv_path, img_dir, img_size=518):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['new_filename'])
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img, row['new_filename']

# 準備測試集
test_dataset = TestDataset('data/test_data_sample.csv', 'data/test_images')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

device = torch.device('cuda')
print(f"✅ 測試集: {len(test_dataset)} 樣本")
print(f"✅ Device: {device}\n")

# 收集所有 fold 的預測
all_preds = []
filenames = None

for fold in range(5):
    model_path = f'outputs/dinov2_colab/fold{fold}/best.pt'
    
    if not os.path.exists(model_path):
        print(f"⚠️ Fold {fold} 模型不存在，跳過")
        continue
    
    print(f"📊 Fold {fold} 預測...")
    
    # 載入模型
    model = timm.create_model('vit_base_patch14_dinov2.lvd142m', 
                              pretrained=False, num_classes=4)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    # 預測
    fold_probs = []
    fold_filenames = []
    
    with torch.no_grad():
        for images, fnames in tqdm(test_loader, desc=f'Fold {fold}', leave=False):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            fold_probs.append(probs.cpu().numpy())
            fold_filenames.extend(fnames)
    
    fold_probs = np.concatenate(fold_probs, axis=0)
    all_preds.append(fold_probs)
    
    if filenames is None:
        filenames = fold_filenames
    
    print(f"   ✅ 完成: {fold_probs.shape}")

# 檢查是否有任何預測
if len(all_preds) == 0:
    print("\n❌ 錯誤: 沒有找到任何訓練好的模型")
    print("   請確保至少訓練了一個 Fold")
else:
    # 平均所有 fold 的預測
    print(f"\n📊 集成 {len(all_preds)} 個模型的預測...")
    avg_probs = np.mean(all_preds, axis=0)
    final_preds = np.argmax(avg_probs, axis=1)
    
    # 創建提交文件
    class_names = ['normal', 'bacteria', 'virus', 'COVID-19']
    submission_df = pd.DataFrame({
        'new_filename': filenames,
        'label': [class_names[p] for p in final_preds]
    })
    
    # 保存
    submission_path = f'submission_dinov2_{len(all_preds)}fold.csv'
    submission_df.to_csv(submission_path, index=False)
    
    print(f"\n✅ 集成預測已保存: {submission_path}")
    print(f"\n預測分布:")
    print(submission_df['label'].value_counts())
    print(f"\n{'='*70}")
    print("準備提交到 Kaggle！")
    print("="*70)

## 1️⃣1️⃣ 提交到 Kaggle 並查看結果

In [ ]:
import os
import glob

# 找到最新的提交文件
submission_files = glob.glob('submission_dinov2_*fold.csv')
if submission_files:
    latest_submission = max(submission_files, key=os.path.getctime)
    
    print(f"📤 提交文件: {latest_submission}")
    print()
    
    # 提交
    !kaggle competitions submit \
        -c cxr-multi-label-classification \
        -f {latest_submission} \
        -m "DINOv2 Ensemble (Google Colab A100) - Target 90%+"
    
    print("\n" + "="*70)
    print("✅ 提交完成！")
    print("="*70)
    
    # 查看最近提交
    print("\n最近的提交記錄:")
    !kaggle competitions submissions -c cxr-multi-label-classification | head -10
    
    print("\n" + "="*70)
    print("🎯 等待 Kaggle 評分...")
    print("   通常需要 5-10 分鐘")
    print("   可以前往 Kaggle 競賽頁面查看結果")
    print("="*70)
else:
    print("❌ 找不到提交文件")
    print("   請確保已執行預測生成 Cell")

## 1️⃣2️⃣ (可選) 保存結果到 Google Drive

In [ ]:
from google.colab import drive
import shutil
import os

print("💾 保存結果到 Google Drive...\n")

# 掛載 Google Drive
drive.mount('/content/drive')

# 創建保存目錄
save_dir = '/content/drive/MyDrive/CXR_DINOv2_Results'
os.makedirs(save_dir, exist_ok=True)

print(f"\n保存目錄: {save_dir}\n")

# 複製模型檢查點（可能很大，謹慎）
if os.path.exists('outputs/dinov2_colab'):
    print("📦 複製模型檢查點 (可能需要幾分鐘)...")
    shutil.copytree('outputs/dinov2_colab', 
                   f'{save_dir}/models', 
                   dirs_exist_ok=True)
    print("   ✅ 模型已保存")

# 複製提交文件
for f in glob.glob('submission_*.csv'):
    shutil.copy(f, save_dir)
    print(f"   ✅ {f}")

print(f"\n✅ 所有結果已保存到 Google Drive")
print(f"   位置: {save_dir}")

# 顯示保存的文件
!ls -lh {save_dir}

---

## 📊 訓練完成總結

### ✅ 完成檢查清單

- [ ] 環境檢查通過（GPU: A100）
- [ ] 所有依賴安裝成功
- [ ] Kaggle API 設置完成
- [ ] GitHub 倉庫 clone 成功
- [ ] 數據集下載完成（3.4 GB）
- [ ] Fold 數據準備完成
- [ ] Fold 0 驗證完成（Val F1 ≥ 88%）
- [ ] 5-Fold 完整訓練完成
- [ ] 集成預測生成
- [ ] 提交到 Kaggle
- [ ] 結果保存到 Google Drive

---

### 🎯 預期結果

- **當前最佳**: 87.574%
- **DINOv2 目標**: 89.5-91.5%
- **突破目標**: 90%+

---

### 📝 下一步

#### 如果達到 90%+ ✅
1. 嘗試 DINOv2-Large 模型（更大）
2. 與現有最佳模型集成
3. 探索 Test-Time Augmentation

#### 如果未達標 (< 89%) ⚠️
1. 檢查訓練日誌，分析問題
2. 調整超參數（learning rate, batch size）
3. 嘗試其他方案（見 BREAKTHROUGH_STRATEGY_ANALYSIS.md）

---

**項目倉庫**: https://github.com/thc1006/nycu-CSIC30014-LAB3

**完整文檔**: 見 CLAUDE.md 和 MACHINE_HANDOFF_GUIDE.md

**祝您突破 90%！** 🚀
